<a href="https://colab.research.google.com/github/kevinOriginal/news-recommender/blob/main/NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kill -9 -1

In [ ]:
from glob import glob
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
import os, sys
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/mnt',force_remount=True)

path = '/content/mnt/My Drive/Colab Notebooks/user_vector.parquet'
path2 = '/content/mnt/My Drive/Colab Notebooks/data-frame.parquet'
path3 = '/content/mnt/My Drive/Colab Notebooks/comments_index.csv'


Mounted at /content/mnt


In [ ]:
# Read user_vector.parquet
df = pd.read_parquet(path) 
print("dtype: ", df.to_numpy().dtype)
# news_id_list = df.columns
# user_id_list = df.index.values
# print(user_id_list)
# print(df.head())

dtype:  uint8


In [ ]:
# news_df = pd.read_csv(path3)
news_df = pd.read_parquet(path2)
# news_df = news_df.to_numpy()
# print("dtype: ", news_df.dtype)
# news_df.astype(str)
# print("New dtype: ", news_df.dtype)
# news_df.userId.apply(str)
# news_df.objectId.apply(str)
# news_df.sympathyCount.apply(int)
# news_df.antipathyCount.apply(int)


# print(shit)
news_df['marked'] = 1

news_df = news_df.reset_index()[['userId', 'objectId', 'sympathyCount', 'antipathyCount', 'marked']]
news_df['marked'] = 1
news_df_arr = news_df.to_numpy()
news_df_arr.astype(np.string_)
# news_df['userId'].astype(np.string_)
# news_df['objectId'].astype(np.string_)

print(news_df.head())
print(news_df.columns)

      userId            objectId  sympathyCount  antipathyCount  marked
0  rudd1kmeI  news008,0004390397           8564              91       1
1  edmo28lVR  news008,0004390397           5340              84       1
2  jinh3V2DH  news008,0004390397           4757             992       1
3  jgb72DP5v  news008,0004390397           2193              46       1
4  paki2eFia  news008,0004390397           1926              63       1
Index(['userId', 'objectId', 'sympathyCount', 'antipathyCount', 'marked'], dtype='object')


In [ ]:
# data = torch.DoubleTensor(df.values)
# print("전체 데이터 : ", data.shape)
# print("shape[0] : ", data.shape[0])


In [ ]:
train_df, valid_df = train_test_split(news_df, test_size=0.2)

train_df = train_df.reset_index()[['userId', 'objectId', 'sympathyCount', 'antipathyCount', 'marked']]
valid_df = valid_df.reset_index()[['userId', 'objectId', 'sympathyCount', 'antipathyCount', 'marked']]

# train_df.head()
valid_df.head()

,userId,objectId,sympathyCount,antipathyCount,marked
0,kmjn2jbJX,"news005,0001310058",5,18,1
1,spea7QAMT,"news008,0004394376",0,0,1
2,ehle1wNG6,"news025,0002991962",0,0,1
3,insonAT3,"news001,0011542145",0,0,1
4,rara9gh8r,"news025,0002992089",0,0,1


In [ ]:
def encode_column(column):
    """ Encodes a pandas column with continous IDs"""
    keys = column.unique()
    key_to_id = {key:idx for idx,key in enumerate(keys)}
    return key_to_id, np.array([key_to_id[x] for x in column]), len(keys)

def encode_df(df):
    """Encodes rating data with continuous user and news ids"""
    
    news_ids, df['objectId'], num_news = encode_column(df['objectId'])
    user_ids, df['userId'], num_users = encode_column(df['userId'])
    return df, num_users, num_news, user_ids, news_ids

  

In [ ]:
train_df, num_users, num_news, user_ids, news_ids = encode_df(train_df)

Y = create_sparse_matrix(train_df, num_users, num_news)
print("Number of users :", num_users)
print("Number of news :", num_news)
print("userid values", train_df['userId'].values)
print("userid values", train_df['objectId'].values)



Y.todense()

Number of users : 326260
Number of news : 1422
userid values [     0      1      2 ... 112765 182911  14284]
userid values [  0   1   2 ... 592 725 831]


matrix([[1, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
def create_sparse_matrix(df, rows, cols, column_name = 'marked'):
    """ Returns a sparse utility matrix""" 
    val_1 = df['userId'].values
    val_1.astype('str')
    val_2 = df['objectId'].values
    val_2.astype('str')
    val_0 = df[column_name].values
    val_0.astype('str')
    return sparse.csc_matrix((val_0, (val_1, val_2)),shape=(rows, cols), dtype='string')


def create_embeddings(n, K):
    """
    Creates a random numpy matrix of shape n, K with uniform values in (0, 11/K)
    n: number of items/users
    K: number of factors in the embedding 
    """
    return 11*np.random.random((n, K)) / K

# emb_sample = create_embeddings(1422, 3)
# print(emb_sample.shape[0])
# print(emb_sample.shape[1])

In [ ]:
lmbda = 0.0002


def predict(df, emb_user, emb_news):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Computes df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    df['prediction'] = np.sum(np.multiply(emb_news[df['objectId']],emb_user[df['userId']]), axis=1)
    return df
    

def cost(df, emb_user, emb_news):
    """ Computes mean square error"""
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_news.shape[0])
    predicted = create_sparse_matrix(predict(df, emb_user, emb_news), emb_user.shape[0], emb_news.shape[0], 'prediction')
    return np.sum((Y-predicted).power(2))/df.shape[0] 





In [ ]:
def gradient(df, emb_user, emb_news):
    """ Computes the gradient for user and news embeddings"""
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_news.shape[0])
    predicted = create_sparse_matrix(predict(df, emb_user, emb_news), emb_user.shape[0], emb_news.shape[0], 'prediction')
    delta =(Y-predicted)
    grad_user = (-2/df.shape[0])*(delta*emb_news) + 2*lmbda*emb_user
    grad_news = (-2/df.shape[0])*(delta.T*emb_news) + 2*lmbda*emb_news
    return grad_user, grad_news


def gradient_descent(df, emb_user, emb_news, iterations=2000, learning_rate=0.01, df_val=None):
    """ 
    Computes gradient descent with momentum (0.9) for given number of iterations.
    emb_user: the trained user embedding
    emb_news: the trained news embedding
    """
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_news.shape[0])
    beta = 0.9
    grad_user, grad_news = gradient(df, emb_user, emb_news)
    v_user = grad_user
    v_news = grad_news
    print("shit")
    for i in range(iterations):
        grad_user, grad_news = gradient(df, emb_user, emb_news)
        v_user = beta*v_user + (1-beta)*grad_user
        v_news = beta*v_news + (1-beta)*grad_news
        emb_user = emb_user - learning_rate*v_user
        emb_news = emb_news - learning_rate*v_news
        if(not (i+1)%50):
            print("\niteration", i+1, ":")
            print("train mse:",  cost(df, emb_user, emb_news))
            if df_val is not None:
                print("validation mse:",  cost(df_val, emb_user, emb_news))
    return emb_user, emb_news

In [ ]:
print("user num", num_users)
print("news num", num_news)

emb_user = create_embeddings(num_users, 3)
emb_news = create_embeddings(num_news, 3)

Y = create_sparse_matrix(news_df, emb_user.shape[0], emb_news.shape[0])


emb_user, emb_news = gradient_descent(news_df, emb_user, emb_news, iterations=800, learning_rate=1)


NameError: ignored

In [ ]:
print("Test")

Test
